In [ ]:
# Install libraries (first time)
%pip install opencv-python mediapipe

In [ ]:
# Import libraries
import time
import numpy as np 
import cv2 
import mediapipe as mp 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose 

In [ ]:
# Functions
def calculateAngle(a,b,c):
    a = np.array(a) # start
    b = np.array(b) # centre
    c = np.array(c) # end
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

def track_exercise(exercise, mode):

    cap = cv2.VideoCapture(0)

    counter = 0
    stage = "Down"
    warning = "" 
    timer = 60
    prev = False

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

        while cap.isOpened():
            ret, frame = cap.read() 
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False 
            results = pose.process(image)
            image.flags.writeable = True 
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            try:
                landmarks = results.pose_landmarks.landmark 
                shoulder = [landmarks[11].x, landmarks[11].y]
                elbow = [landmarks[13].x, landmarks[13].y]
                wrist = [landmarks[15].x, landmarks[15].y]
                hip = [landmarks[23].x, landmarks[23].y]
                knee = [landmarks[25].x, landmarks[25].y]
                ankle = [landmarks[27].x, landmarks[27].y]
                
                arm_angle = calculateAngle(shoulder,elbow,wrist)
                back_angle = calculateAngle(shoulder,hip,knee)
                situp_angle = calculateAngle(shoulder,hip,ankle)
                
                if exercise == "pushup":
                    cv2.putText(image, str(round(arm_angle, 2)), tuple(np.multiply(elbow, [cap.get(cv2.CAP_PROP_FRAME_WIDTH),cap.get(cv2.CAP_PROP_FRAME_HEIGHT)]).astype(int)), #[width,height] - webcam default dimension
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                    cv2.putText(image, str(round(back_angle, 2)), tuple(np.multiply(hip, [640,480]).astype(int)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                    if back_angle < 150 and back_angle > 30:
                        warning = "Back"
                    else:
                        warning = ""
                        
                    if arm_angle < 80: 
                        stage = "Up"
                    
                    if arm_angle > 160 and stage == "Up": 
                        stage = "Down"
                        if warning=="":
                            counter += 1
                            if counter == 1 and mode == "test":
                                prev = time.time()

                elif exercise == "situp":
                    cv2.putText(image, str(round(situp_angle, 2)), tuple(np.multiply(hip, [640,480]).astype(int)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                    if back_angle > 170:
                        stage = "Up"
                    if back_angle < 120 and stage == "Up":
                        stage = "Down"
                        counter += 1
                        if counter == 1 and mode == "test":
                            prev = time.time()
            
            except:
                pass
            
            if prev:
                cur = time.time()
                if cur-prev >= 1:
                    prev = cur
                    timer -= 1
                
            if timer == 0:
                break
                
            cv2.rectangle(image, (0,0), (300,140), (245,117,16), -1)
            cv2.rectangle(image, (540,0), (640,100), (255, 255, 255), -1)
            
            cv2.putText(image, "Reps:", (15,30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (10,100),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 2, cv2.LINE_AA)
            cv2.putText(image, str(timer), (550,70),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2, cv2.LINE_AA)
            
            cv2.putText(image, "Stage:", (120,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
            cv2.putText(image, str(stage), (120,75),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
            cv2.putText(image, str(warning), (120, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1, cv2.LINE_AA)
            
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
            
            cv2.imshow("Mediapipe Feed", image) 
            
            if (cv2.waitKey(10) & 0xFF == ord('q')) or cv2.getWindowProperty("Mediapipe Feed", cv2.WND_PROP_VISIBLE)<1: 
                break

    cap.release()
    cv2.destroyAllWindows()
    return counter

In [ ]:
# Main
print("IPPT Exercises Form Tracker\n1. Push-up\n2. Sit-up")
exercise = int(input("Select exercise: "))

while exercise == 1 or exercise == 2:
    if exercise == 1:
        print("1. Practice\n2. Test (60 seconds)")
        option = int(input("Select mode: "))
        if option == 1:
            print("Enter 'q' to quit")
            counter = track_exercise("pushup", "prac")
        elif option == 2:
            counter = track_exercise("pushup", "test")
        print(f"{counter} push-up(s).")

    elif exercise == 2:
        print("Start from upright pos\n1. Practice\n2. Test (60 seconds)")
        option = int(input("Select mode: "))
        if option == 1:
            counter = track_exercise("situp", "prac")
        elif option == 2:
            counter = track_exercise("situp", "test")
        print(f"{counter} sit-up(s).")

    exercise = int(input("Select exercise: "))

print("Programme exited.")